# About this worksheet

In the interest of making this a fantastic competition for all of us, and to give back to others who have also made their work public, I decided to create a core worksheet of my work to help everyone get started. Just knowing that this is the model that's currently on the leaderboard should give you a little confidence in it.  

I still want this to be a competition, so I'm not giving away the family jewels. This is not the tuned model that is currently in second place, but it's most of it.  So don't expect to get a good score as is. But with a little tweaking and some outside-the-box thinking, it *can* get you onto the leaderboard (and maybe knock me off of it.)
  
I apologize in advance if my comments on these cells are below your skill level.  I definitely don't want to insult anyone's intelligence, but I decided to make things as basic as possible to help even beginners get started.  
  
Be sure to tip the bartender and your waiters/waitresses, and upvote any of the public worksheets/discussions that you find useful.  I've already upvoted a few that helped me, especially the one by Tarek Hamdi.  A little respect and appreciation goes a long way. 

And I want to thank Jane Street and Kaggle for making this competition available to all of us. This is a really fun event.  If you haven't been to the Jane Street website, I encourage a quick look.  It seems like a fun and worthwhile place to work.  I especially like their workplace values that they mention on the [*Join Jane Street*](https://www.janestreet.com/join-jane-street/) page.

Thanks, everyone, and good luck!


# Load libraries and create the competition environment
All imports up-front for easy reference.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import cudf

from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

import pandas as pd
pd.set_option('display.max_columns', 500)

import xgboost as xgb
print("XGBoost version:", xgb.__version__)

import warnings
warnings.filterwarnings("ignore")

# create the environment
import janestreet
print('Creating competition environment...', end='')
env = janestreet.make_env()
print('Finished.')

# Read the data  
Uses cudf to speed the process a few seconds as opposed to a straight read_csv into Pandas. Feel free to uncomment and use the Pandas version instead.

In [ ]:
print('Reading datasets...', end='')

#train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train_cudf = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train = train_cudf.to_pandas()
del train_cudf

#features_meta = pd.read_csv('/kaggle/input/jane-street-market-prediction/features.csv')
features_meta_cudf = cudf.read_csv('/kaggle/input/jane-street-market-prediction/features.csv')
features_meta = features_meta_cudf.to_pandas()
del features_meta_cudf

#example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
#sample_prediction_df = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
print('Finished.')

print(f'train shape: {format(train.shape)}')
print(f'features_meta shape: {format(features_meta.shape)}')
#print(f'example_test shape: {format(example_test.shape)}')
#print(f'sample_prediction_df shape:{format(sample_prediction_df.shape)}')

# display the first rows of the training data
# print(train.head())

# Preprocess the data
Create the list of features to use for training; either the entire set or a subset based on importance. Set the target to a binary value based on its sign.   

You can replace missing fields with either a unique constant (like -99999) or some other value (generally the mean of that feature from all the other rows with valid data), or you can leave them as NaN's. If the means are used, it's good to retain them for use later when the model is tested so they can also replace any missing data coming in from the test enviroment. The XGBoost model supports specifying a constant for a missing value, so the means can still be used for training and the constant or None used during testing.  

Try each and see what works best.

In [ ]:
# preprocess data and create training dataset

print('Preprocessing data...', end='')

# create list of training feature column names
features = [col for col in list(train.columns) if 'feature' in col]

# # adjust features to use top half based on importance
# features = ['feature_43', 'feature_42', 'feature_45', 'feature_41', 'feature_44', 'feature_63', 'feature_61', 'feature_69', 'feature_6', 'feature_64', 'feature_5', 'feature_62', 'feature_7', 'feature_11', 'feature_20', 'feature_39', 'feature_60', 'feature_83', 'feature_3', 'feature_37', 'feature_1', 'feature_40', 'feature_4', 'feature_38', 'feature_27', 'feature_77', 'feature_119', 'feature_28', 'feature_120', 'feature_68', 'feature_95', 'feature_90', 'feature_66', 'feature_55', 'feature_121', 'feature_89', 'feature_84', 'feature_107', 'feature_114', 'feature_113', 'feature_71', 'feature_8', 'feature_124', 'feature_49', 'feature_101', 'feature_125', 'feature_102', 'feature_78', 'feature_57', 'feature_67', 'feature_65', 'feature_108', 'feature_70', 'feature_31', 'feature_48', 'feature_126', 'feature_18', 'feature_26', 'feature_96', 'feature_86', 'feature_116', 'feature_127', 'feature_22', 'feature_92', 'feature_51', 'feature_58', 'feature_12', 'feature_33', 'feature_53', 'feature_17', 'feature_104', 'feature_24', 'feature_110', 'feature_72', 'feature_36', 'feature_21', 'feature_35', 'feature_32', 'feature_25', 'feature_59', 'feature_34', 'feature_2', 'feature_93', 'feature_10', 'feature_117', 'feature_87', 'feature_47', 'feature_128', 'feature_98', 'feature_80', 'feature_50', 'feature_54', 'feature_79', 'feature_129', 'feature_9', 'feature_23', 'feature_19', 'feature_111', 'feature_115', 'feature_56', 'feature_30', 'feature_73', 'feature_122', 'feature_105', 'weight', 'feature_74', 'feature_14', 'feature_123', 'feature_29', 'feature_109', 'feature_82', 'feature_112', 'feature_76', 'feature_16', 'feature_99', 'feature_88', 'feature_85', 'feature_106', 'feature_118', 'feature_91', 'feature_46', 'feature_81', 'feature_75', 'feature_94', 'feature_52', 'feature_13', 'feature_103', 'feature_15', 'feature_100', 'feature_97', 'feature_0']
# features = features[:65]

# only train on rows with positive weights
train = train[train['weight'] != 0]

# binarize the target
train['action'] = (train['resp'].values > 0).astype(int)

#train = train.fillna(-99999)
f_mean = train.mean()
train.fillna(f_mean)

# split data for training and free data space usage to prevent exceeding maximum allowed
X_train = train.loc[:, features]
y_train = train.loc[:, 'action']
del train

print('Finished.')

# Create and configure the model
This is where most of the data science work is done. Tuning the various parameters to get the best results. Experiment with all that the model has available.   

This link is a good reference:
[https://xgboost.readthedocs.io/en/latest/parameter.html](http://)

In [ ]:
#   X G B o o s t
#
# create, configure and train the classifier using GPU
# TODO: other tree_methods ?

print('Creating classifier...', end='')
clf = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=7,
    eta=0.5, # learning_rate
    missing=None,
    random_state=42,
    tree_method='gpu_hist',
    subsample=0.8,
    colsample_bytree=1,
    #sampling_method='gradient_based',
    #eval_metric='logloss',
    verbosity=2   # info
)
print('Finished.')

# Train the model
Make sure you have the GPU accelerator turned on and just sit back for a few minutes.  

If the GPU is not turned on, just click the 3-dot menu at the top right of the session window and then select Accelerator. GPU time is a precious resource, so you're only allowed a certain amount of time each week.  

Be sure to shut the GPU off or stop the session to prevent using up your time.

In [ ]:
# train
print('Training classifier...', end='')
%time clf.fit(X_train, y_train)
print('Finished.')

# Test the model in the environment and create the submission file
* When you have a model ready to go, just click the Save Version button at the top right. It saves a snapshot of the worksheet and runs it again in a separate background session to create the submission file. 

* If you have the patience to watch it complete (usually takes a while, because it runs the entire worksheet again), you'll see the small white pop-up window at the bottom left briefly offer a View button (it times out after a bit and disappears) that creates a new tab to see your worksheet results. 
* Scroll down and you'll see your Submission file with a button on the right to submit the results to the competition. 
* If you miss that little window, just go back to your competition Overview and click Submit Predictions on the right.
* Good luck!  

In [ ]:
# perform test and create submissions file
print('Creating submissions file...', end='')
rcount = 0
for (test_df, prediction_df) in env.iter_test():
    X_test = test_df.loc[:, features]
    y_preds = clf.predict(X_test)
    prediction_df.action = y_preds
    env.predict(prediction_df)
    rcount += len(test_df.index)
print(f'Finished processing {rcount} rows.')

# Experiments
* These cells have a couple of useful tools to assist with configuring the model. They are commented out after use to prevent running them with every submission. 
* Uncomment and experiment. They are in various stages of being operational, so you may need to tweak things.

Adapted from [https://machinelearningmastery.com/tune-number-size-decision-trees-xgboost-python/](http://)  
And from [https://towardsdatascience.com/be-careful-when-interpreting-your-features-importance-in-xgboost-6e16132588e7](http://)

In [ ]:
# #   E X P E R I M E N T S

# # adjust parameters to test fit

# # eval_metric = ['logloss', 'error', 'error@0.6']
# # sampling_method=['uniform', 'gradient_based']
# # param_grid = dict(eval_metric=eval_metric, sampling_method=sampling_method)

# #kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# grid_search = GridSearchCV(clf, param_grid, scoring="neg_log_loss", n_jobs=1, cv=10, verbose=3, refit=False)
# grid_result = grid_search.fit(X_train, y_train)

# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
# 	print("%f (%f) with: %r" % (mean, stdev, param))
# # # plot results
# # scores = np.array(means).reshape(len(eval_metric), len(sampling_method))
# # for i, value in enumerate(sampling_method):
# #     plt.plot(eval_metric, scores[i], label='sampling_method: ' + str(value))
# # plt.legend()
# # plt.xlabel('sampling_method')
# # plt.ylabel('Log Loss')
# # plt.savefig('best_params.png')m

In [ ]:
# # available importance_types = [‘weight’, ‘gain’, ‘cover’, ‘total_gain’, ‘total_cover’]
# f = 'gain'
# top = len(features)  # or set constant to view only the most important
# fdata = clf.get_booster().get_score(importance_type= f)
# fsorted = dict(sorted(fdata.items(), key=lambda item: item[1], reverse=True))
# plt.figure(figsize=(10,top * 0.25))
# plt.barh(range(top), list(fsorted.values())[:top], align='center')
# plt.yticks(range(top), list(fsorted.keys())[:top])
# plt.show()
# #print(list(fsorted.keys()))
# #print(fsorted)